In [11]:
import numpy as np
import rasterio
from scipy import ndimage
import os

In [96]:
import time
import numpy as np
import rasterio
from scipy.ndimage import convolve, median_filter

def pit_fill_chm(chm_path, threshold_percent=5.0, output_path=None):
    """
    Applies pit filling to a CHM (canopy height model) stored in a TIFF file. Methods described in:
    
    Ben-Arie, J. R., Hay, G. J., Powers, R. P., Castilla, G., & St-Onge, B. (2009). 
    Development of a pit filling algorithm for LiDAR canopy height models. Computers & Geosciences, 35(9), 
    1940–1949. https://doi.org/10.1016/j.cageo.2009.02.003

    Parameters:
        chm_path (str): Path to the input CHM GeoTIFF.
        threshold_percent (float): The percentile (0-100) used to select the pit threshold
                                   from the cumulative histogram of the Laplacian image.
                                   (Default: 5.0, i.e., 5%)
        output_path (str): If provided, the pit-filled CHM will be saved to this file.

    Returns:
        chm_filled (np.ndarray): The pit-filled CHM as a numpy array.
    """
    # Start timing the processing
    start_time = time.time()

    # Read the CHM using rasterio
    with rasterio.open(chm_path) as src:
        chm = src.read(1)  # assuming a single band CHM
        profile = src.profile

    # ---------------------------
    # Step 1: Compute Laplacian
    # ---------------------------
    # Define the Laplacian kernel
    kernel = np.array([[-1, -1, -1],
                       [-1,  8, -1],
                       [-1, -1, -1]])
    # Compute the Laplacian using convolution (more appropriate than median_filter)
    lap = convolve(chm, kernel, mode='reflect')

    lap_min = lap.min()
    lap_max = lap.max()

    # ---------------------------
    # Step 2: Threshold selection
    # ---------------------------
    # Compute the threshold value such that 'threshold_percent' of the pixels have a 
    # Laplacian value below this threshold.
    lap_threshold = np.percentile(lap, threshold_percent)

    # ---------------------------
    # Step 3: Generate pit mask
    # ---------------------------
    # Identify pits as pixels where the Laplacian is below the threshold.
    pit_mask = lap < lap_threshold
    num_pits = np.sum(pit_mask)

    # ---------------------------
    # Step 4: Apply median filter to smooth the CHM
    # ---------------------------
    # A median filter is useful for reducing local noise.
    chm_median = median_filter(chm, size=3)

    # ---------------------------
    # Step 5: Pit filling
    # ---------------------------
    # For pixels identified as pits, replace the original CHM value with the median-filtered value.
    chm_filled = chm.copy()
    chm_filled[pit_mask] = chm_median[pit_mask]

    # Count negative values before correction
    num_negative_before = np.sum(chm_filled < 0)

    # ---------------------------
    # Step 6: Correct negative values
    # ---------------------------
    chm_filled[chm_filled < 0] = 0

    processing_time = time.time() - start_time

    # ---------------------------
    # Reporting
    # ---------------------------
    print("Minimum Laplacian value: {}".format(lap_min))
    print("Maximum Laplacian value: {}".format(lap_max))
    print("Laplacian threshold value (at {}%): {}".format(threshold_percent, lap_threshold))
    print("Number of data pits identified: {}".format(num_pits))
    print("Number of negative values before correction: {}".format(num_negative_before))
    print("Processing time: {:.2f} seconds".format(processing_time))

    # Save the pit-filled CHM if an output path is provided.
    if output_path:
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(chm_filled, 1)
        print("Pit-filled CHM saved to:", output_path)
    else:
        print("No output file specified; pit-filled CHM not saved to disk.")

    return chm_filled

In [94]:
input = '/Users/kdoherty/calibrate_rap/data/raster/lidar_processed/chms/2016.tif'
output = f'/Users/kdoherty/calibrate_rap/data/raster/lidar_processed/chms/2016_filled.tif'


In [95]:
pit_fill_chm(input, 5, output)

Minimum Laplacian value: -inf
Maximum Laplacian value: inf
Laplacian threshold value (at 5%): -14.16998291015625
Number of data pits identified: 1810246
Number of negative values before correction: 13196583
Processing time: 1.85 seconds
Pit-filled CHM saved to: /Users/kdoherty/calibrate_rap/data/raster/lidar_processed/chms/2016_filled.tif


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [97]:
#replace 2016.tif with 2016_filled.tif
import os

# Get the directory path
dir_path = '/Users/kdoherty/calibrate_rap/data/raster/lidar_processed/chms'

# Delete the original file
os.remove(os.path.join(dir_path, '2016.tif'))

# Rename the filled file to the original name
os.rename(os.path.join(dir_path, '2016_filled.tif'), 
         os.path.join(dir_path, '2016.tif'))
